In [ ]:
!pip install transformers
!pip install emoji
!pip install googletrans==4.0.0-rc1
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!npm install -g hanspell
!apt-get install -y nodejs npm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=393100f772b188447024fe1e70294648dc681a55710f58cd7ba80e3f01fd3006
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet

npm WARN deprecated har-validator@5.1.5: this library is no longer supported
npm WARN deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
npm WARN deprecated request@2.88.2: request has been deprecated, see https://github.com/request/request/issues/3142

added 64 packages, and audited 65 packages in 4s

2 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

Some issues need review, and may require choosing
a different dependency.

Run `npm audit` for details.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  gyp javascript-common libc-ares2 libjs-events libjs-highlight.js libjs-inherits
  libjs-is-typedarray libjs-psl libjs-source-map libjs-sprintf-js libjs-typedarray-to-buffer
  lib

In [ ]:
import pandas as pd
import re
import emoji
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer, AutoModel
from googletrans import Translator
from pykospacing import Spacing
from tqdm import tqdm

In [ ]:
def correct_space(text):
    spacing = Spacing()
    return spacing(text)

In [ ]:
def clean_emoticons(text):
    text = emoji.replace_emoji(text, replace="")

    emotion_pattern = re.compile(r"!|\?|\~|\.|,|;|\^|\-|_|@|\[|\]|\+||\*|>|<|{|}|\'|\"|%")
    text = emotion_pattern.sub('', text)

    brackets = re.compile(r"[\(\)]")
    text = brackets.sub('', text)

    hangul_pattern = re.compile(r"[ㄱ-ㅎㅏ-ㅣ]")
    text = hangul_pattern.sub('', text)

    return text

In [ ]:
def translate_english_to_korean(text):
    translator = Translator()

    english_pattern = re.compile(r'[a-zA-Z]+')

    def translate_match(match):
        english_text = match.group(0)
        translated = translator.translate(english_text, src='en', dest='ko').text
        return translated

    translated_text = english_pattern.sub(translate_match, text)

    return translated_text

In [ ]:
def remove_short_and_missing(df):
    df_cleaned = df[~(df['sentence_1'].isnull() | df['sentence_2'].isnull())]
    df_cleaned = df_cleaned[df_cleaned['sentence_1'].str.len() > 2]
    df_cleaned = df_cleaned[df_cleaned['sentence_2'].str.len() > 2]

    return df_cleaned

In [ ]:
text = "해외 나갈때 (48시간)내 PCR검사를 원하는 나라가 있는데요, 요즘 코로나 환자가 줄면서 병원에서 PCR검사를 안하는 경우도 많았졌다고 합니다."


check_space = correct_space(text)
print(check_space)
clean_emotic = clean_emoticons(check_space)
print(clean_emotic)
converted_text = translate_english_to_korean(clean_emotic)
print(converted_text)

해외 나갈 때 (48시간) 내 PCR 검사를 원하는 나라가 있는데요, 요즘 코로나 환자가 줄면서 병원에서 PCR 검사를 안하는 경우도 많았졌다고 합니다.
해외 나갈 때 48시간 내 PCR 검사를 원하는 나라가 있는데요 요즘 코로나 환자가 줄면서 병원에서 PCR 검사를 안하는 경우도 많았졌다고 합니다
해외 나갈 때 48시간 내 PCR 검사를 원하는 나라가 있는데요 요즘 코로나 환자가 줄면서 병원에서 PCR 검사를 안하는 경우도 많았졌다고 합니다


In [ ]:
df = pd.read_csv('train_v3_all_swap.csv')
df['sentence_1'].to_csv('input_text1.txt', index=False, header=False)
df['sentence_2'].to_csv('input_text2.txt', index=False, header=False)

In [ ]:
!cat input_text1.txt | hanspell-cli -a > corrected_text1.txt
!cat input_text2.txt | hanspell-cli -a > corrected_text2.txt

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
있지만..좋을수도있다 -> 있지만.. 좋을 수도 있다
뒤에 오는 명사를 수식하는 관형격 어미 ‘-ㄴ’, ‘-는’, ‘-던’, ‘-ㄹ’ 등과 의존명사는 띄어 쓰는 것이 옳습니다.
(예)
노력한 만큼 대가를 얻다.
소문으로만 들었을 뿐이네.
합격했다는 소리를 들으니 그저 기쁠 따름이다.
온것 -> 온 것
뒤에 오는 명사를 수식하는 관형격 어미 ‘-ㄴ’, ‘-는’, ‘-던’, ‘-ㄹ’ 등과 의존명사는 띄어 쓰는 것이 옳습니다.
(예)
노력한 만큼 대가를 얻다.
소문으로만 들었을 뿐이네.
합격했다는 소리를 들으니 그저 기쁠 따름이다.
하시구요." -> 하시고요."
올바르지 않은 어미의 사용입니다. '하시고요."'로 고쳐 씁니다.
없더라구요ㅠ" -> 없더라고요ㅠ"
올바르지 않은 어미의 사용입니다. '없더라고요ㅠ"'으로 고쳐 씁니다.
추천 받아 -> 추천받아
일부 명사 아래에 붙어, ‘그로 인한 결과를 입다’ 또는 ‘그러한 일을 당하다’의 뜻을 더하여 동사를 만드는 접미사 '-받다'는 앞말과 붙여 씁니다.
(예)
그의 노래들은 입에서 입으로 널리 전해져 대중으로부터 사랑받았다.
교육적으로 혜택받지 못한 소외 계층에 대한 관심을 불러일으켰다.
어린 소년이 백혈병으로 고통받고 있다.
웃을듯ㅋㅋ -> 웃을 듯ㅋㅋ
뒤에 오는 명사를 수식하는 관형격 어미 ‘-ㄴ’, ‘-는’, ‘-던’, ‘-ㄹ’ 등과 의존명사는 띄어 쓰는 것이 옳습니다.
(예)
노력한 만큼 대가를 얻다.
소문으로만 들었을 뿐이네.
합격했다는 소리를 들으니 그저 기쁠 따름이다.
재고해주세요 -> 재고해 주세요
보조 용언이 두 개 이상 거듭되는 경우 또는 합성동사 뒤에 오는 보조용언은 띄어 쓰는 게 옳습니다.
(예)
우리나라는 삼면이 바다로 둘러싸여 있다.
이 역사극은 남녀 모두 좋아할 만한 내용이다.
바람에 다 넘어가 버렸네.
가시는건 -> 가시는 건
뒤에 오는 명사를 수식하는 관형격 어미 ‘-ㄴ’, ‘-는’, ‘-던’, ‘-ㄹ’ 등과 의존명사는 띄어

In [ ]:
with open('corrected_text1.txt', 'r') as f:
    corrected_sentences = f.readlines()

df['sentence_1'] = corrected_sentences

with open('corrected_text2.txt', 'r') as f:
    corrected_sentences = f.readlines()

df['sentence_2'] = corrected_sentences

In [ ]:
tqdm.pandas()

df['sentence_1'] = df['sentence_1'].progress_apply(correct_space)
df['sentence_2'] = df['sentence_2'].progress_apply(correct_space)

df['sentence_1'] = df['sentence_1'].progress_apply(clean_emoticons)
df['sentence_2'] = df['sentence_2'].progress_apply(clean_emoticons)

df['sentence_1'] = df['sentence_1'].progress_apply(translate_english_to_korean)
df['sentence_2'] = df['sentence_2'].progress_apply(translate_english_to_korean)

df = remove_short_and_missing(df)

print(df.head())

df.to_csv('train_v3_all_swap_corrected.csv', index=False)

100%|██████████| 23310/23310 [31:03<00:00, 12.51it/s]


                            id            source  \
0   boostcamp-sts-v1-train-137         slack-rtt   
1  boostcamp-sts-v1-train-4176  petition-sampled   
2  boostcamp-sts-v1-train-3296  petition-sampled   
3  boostcamp-sts-v1-train-7437      petition-rtt   
4  boostcamp-sts-v1-train-1068  petition-sampled   

                                 sentence_1  \
0                        평화로 어딘가 일 경우 한 수위    
1                         국회의원 피감기관 후원 해외출장   
2                        크림하우스 매트 조사 부탁드립니다   
3  그리고 그런 김정은은 다시 한번 본심을 드러내 돌변할지도 모르는 법입니다   
4                             천안함 사건 재 조사청원   

                                    sentence_2  label  binary-label  
0                                 평화로운 곳이라면 최고    2.5             1  
1          피감기관 후원 해외출장 국회의원 전수조사 & 위법 시 법적 처리    1.5             0  
2                            크림하우스 매트 관련 청원합니다    1.5             0  
3  그리고 그런 김정은이 자신의 진짜 의도와 변화를 다시 한 번 드러낼 수도 있다    3.5             1  
4                         천안함의 정말 중요한 의혹 조사 청원    3.5 

In [ ]:
df = pd.read_csv('dev.csv')
df['sentence_1'].to_csv('input_text1.txt', index=False, header=False)
df['sentence_2'].to_csv('input_text2.txt', index=False, header=False)

In [ ]:
!cat input_text1.txt | hanspell-cli -a > corrected_text1.txt
!cat input_text2.txt | hanspell-cli -a > corrected_text2.txt

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
하지마세요 -> 하지 마세요
연결어미 '-아/-어/-여' 뒤에 나오는 보조용언은 띄어 씀을 원칙으로 하지만, 붙여 씀도 허용합니다. 단, 보조용언 '-어지다.'는 항상 본용언에 붙여 씁니다. 그러나 '-아/-어/-여' 이외의 어미 뒤에 보조용언이 나오거나 본용언이 올 때는 띄어 써야 합니다.
사형 해야합니다 -> 사형해야 합니다
'해야 하다.'로 띄어 씁니다.
레몬워터 -> 레몬 물, 레몬 식수, 레몬 용액
외래어보다는 순수 우리말이나 한자어로 된, 우리가 자주 쓰는 말을 사용합시다.
넵 -> 네
흔히 말을 함부로 변형하거나, 띄어쓰기를 바르게 하지 않는 등 잘못 사용하는 경향이 있습니다. 바른 표현을 사용해야 합니다.
github에서 -> git hub에서
영어 단어를 잘못 표기하셨습니다.
고유명사의 첫 글자나 문장을 시작할 때는 대문자로 써야 하고, 다른 글자나 일반 명사는 소문자로 적습니다. 첫 글자만 따서 만든 단어일 때는 모두 대문자로 씁니다.
(예)
Seoul
appletiME -> time
로마자를 입력할 때도 한글처럼 알파벳을 잘못 입력하여 오류를 범할 수 있으므로 주의해야 합니다.
(1) 알파벳의 순서를 바꿔 쓰거나 다른 글자를 입력한 오류
adn -> and
theone -> throne
(2) 필요한 글자가 빠지거나 불필요한 글자를 더 입력한 오류
comitment -> commitment
wellcome -> welcome
관리중인데 -> 관리 중인데
동작 명사 뒤에 오는 '중'과 '시'를 접미사로 보는 학자도 있습니다만 표준국어대사전[국립국어원]은 의존명사로 봅니다. 이 시스템은 되도록 표준국어대사전[국립국어원]을 따릅니다. 또한 '시'는 되도록 '때'로 고쳐 쓰면 좋습니다.
러닝타임이 -> 상영시간이, 공영시간이, 달린 시간이
외래어보다는 순수 우리말이나 한자어로 된, 우리가 자주 쓰는 말을 사용합시다.
맥어보이가 -> 매커보이가
'McAvoy'를 쓰는 방법은 정해지지 않

In [ ]:
with open('corrected_text1.txt', 'r') as f:
    corrected_sentences = f.readlines()

df['sentence_1'] = corrected_sentences

with open('corrected_text2.txt', 'r') as f:
    corrected_sentences = f.readlines()

df['sentence_2'] = corrected_sentences

In [ ]:
df['sentence_1'] = df['sentence_1'].progress_apply(correct_space)
df['sentence_2'] = df['sentence_2'].progress_apply(correct_space)

df['sentence_1'] = df['sentence_1'].progress_apply(clean_emoticons)
df['sentence_2'] = df['sentence_2'].progress_apply(clean_emoticons)

df['sentence_1'] = df['sentence_1'].progress_apply(translate_english_to_korean)
df['sentence_2'] = df['sentence_2'].progress_apply(translate_english_to_korean)

df = remove_short_and_missing(df)

print(df.head())

df.to_csv('dev_corrected.csv', index=False)

100%|██████████| 550/550 [00:44<00:00, 12.32it/s]


                         id         source                      sentence_1  \
0  boostcamp-sts-v1-dev-000   nsmc-sampled            액션은 개뿔 총 몇 번 쏘고 끝입니다   
1  boostcamp-sts-v1-dev-001      slack-rtt                    감격스러워 입 막으심    
2  boostcamp-sts-v1-dev-002       nsmc-rtt            이번 연도에 본 영화 중 최악의 영화   
3  boostcamp-sts-v1-dev-003      slack-rtt  특히 평소 음악 채널에 많은 영감을 불어 넣어 주시는    
4  boostcamp-sts-v1-dev-004  slack-sampled                 다음 밥 무대가 기대됩니다    

                        sentence_2  label  binary-label  
0  액션은 흉내만 내 고 그마저도 후반부에는 느린 동작 처리    2.0           0.0  
1                   너무 감동해서 입 다 물어    3.4           1.0  
2                     올해 본 영화 중 최악    4.0           1.0  
3      특히 당신은 항상 많은 음악 채널에 영감을 줍니다    3.4           1.0  
4                     다음 후기도 기대됩니다    1.4           0.0  


In [ ]:
df = pd.read_csv('test.csv')
df['sentence_1'].to_csv('input_text1.txt', index=False, header=False)
df['sentence_2'].to_csv('input_text2.txt', index=False, header=False)

In [ ]:
!cat input_text1.txt | hanspell-cli -a > corrected_text1.txt
!cat input_text2.txt | hanspell-cli -a > corrected_text2.txt

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
삼류로가던가.. -> 삼류로 가던가..
띄어쓰기 오류입니다. 대치어를 참고하여 고쳐 쓰세요.
강화하여주세요. -> 강화하여 주세요.
보조 용언이 두 개 이상 거듭되는 경우 또는 합성동사 뒤에 오는 보조용언은 띄어 쓰는 게 옳습니다.
(예)
우리나라는 삼면이 바다로 둘러싸여 있다.
이 역사극은 남녀 모두 좋아할 만한 내용이다.
바람에 다 넘어가 버렸네.
영화인듯!" -> 영화인 듯!"
뒤에 오는 명사를 수식하는 관형격 어미 ‘-ㄴ’, ‘-는’, ‘-던’, ‘-ㄹ’ 등과 의존명사는 띄어 쓰는 것이 옳습니다.
(예)
노력한 만큼 대가를 얻다.
소문으로만 들었을 뿐이네.
합격했다는 소리를 들으니 그저 기쁠 따름이다.
좋습니당 -> 좋습니다
올바르지 않은 어미의 사용입니다. '좋습니다'로 고쳐 씁니다.
(예)
공사는 예정대로 잘 진행되고 있습니다.
아이를 봐 줄 사람을 찾고 있습니다.
오늘 신문에 공개되지 않은 내용을 처음으로 발표하겠습니다.
주십쇼 -> 주십시오
올바르지 않은 어미의 사용입니다. '주십시오'로 고쳐 씁니다.
국내스릴러중엔나쁘지않은편... -> 국내스릴러중엔나쁘지 않은 편...
뒤에 오는 명사를 수식하는 관형격 어미 ‘-ㄴ’, ‘-는’, ‘-던’, ‘-ㄹ’ 등과 의존명사는 띄어 쓰는 것이 옳습니다.
(예)
노력한 만큼 대가를 얻다.
소문으로만 들었을 뿐이네.
합격했다는 소리를 들으니 그저 기쁠 따름이다.
녹여내보겠습니다. -> 녹여내 보겠습니다.
보조 용언이 두 개 이상 거듭되는 경우 또는 합성동사 뒤에 오는 보조용언은 띄어 쓰는 게 옳습니다.
(예)
우리나라는 삼면이 바다로 둘러싸여 있다.
이 역사극은 남녀 모두 좋아할 만한 내용이다.
바람에 다 넘어가 버렸네.
부활시켜주십시요. -> 부활시켜 주십시오.
보조 용언이 두 개 이상 거듭되는 경우 또는 합성동사 뒤에 오는 보조용언은 띄어 쓰는 게 옳습니다.
(예)
우리나라는 삼면이 바다로 둘러싸여 있다.
이 역사극은 남녀 모두 좋아할 만한 내

In [ ]:
with open('corrected_text1.txt', 'r') as f:
    corrected_sentences = f.readlines()

df['sentence_1'] = corrected_sentences

with open('corrected_text2.txt', 'r') as f:
    corrected_sentences = f.readlines()

df['sentence_2'] = corrected_sentences

In [ ]:
df['sentence_1'] = df['sentence_1'].progress_apply(correct_space)
df['sentence_2'] = df['sentence_2'].progress_apply(correct_space)

df['sentence_1'] = df['sentence_1'].progress_apply(clean_emoticons)
df['sentence_2'] = df['sentence_2'].progress_apply(clean_emoticons)

df['sentence_1'] = df['sentence_1'].progress_apply(translate_english_to_korean)
df['sentence_2'] = df['sentence_2'].progress_apply(translate_english_to_korean)

df = remove_short_and_missing(df)

print(df.head())

df.to_csv('test_corrected.csv', index=False)

100%|██████████| 1100/1100 [01:25<00:00, 12.84it/s]

                          id            source  \
0  boostcamp-sts-v1-test-000  petition-sampled   
1  boostcamp-sts-v1-test-001  petition-sampled   
2  boostcamp-sts-v1-test-002      petition-rtt   
3  boostcamp-sts-v1-test-003      nsmc-sampled   
4  boostcamp-sts-v1-test-004         slack-rtt   

                                      sentence_1  \
0                               가상화폐 거래소 폐쇄하지 말고   
1  뇌물 적폐 1호 640만 달러 70억 뇌물 받은 권 양숙 구속하고 재산을 몰수하라   
2            기무사 영관급의 하극상 정말 이대로 방관하는 게 민주주의 인지요   
3                                      화까지가 한계였다   
4                                      왜 혼자 있지…    

                                       sentence_2  
0                               가상화폐 거래소 폐쇄 반대합니다  
1  뇌물 적폐 원조 640만 달러 70억 뇌물 받은 권 양숙 구속하고 재산을 몰수하세요  
2                          그냥 가만히 있는 게 진짜 민주주의인가요  
3                                      기대가 너무 컸다…  
4                                        왜 혼자 야…   
